Se optó por desarrollar el patrón de diseño *Command* ya que permite la utilización de las heurísticas seleccionadas en diversas meta-heurísticas.

In [22]:
from abc import ABCMeta, abstractmethod

class Heuristic(metaclass=ABCMeta):
    @abstractmethod
    def perturb_solution(self, current_solution):
        pass

In [41]:
class KMoves(Heuristic):
    def __init__(self, k=1, displacements=1):
        self.k = k
        self.displacements = displacements
        
    def _do_displacements(self, graph, moves, term, machine_index, operation_index):
        for _ in range(moves):
            operation = graph[machine_index][operation_index]
            t_operation = graph[machine_index][operation_index + term]
            
            self.m_graph[machine_index][operation_index] = t_operation
            self.m_graph[machine_index][operation_index + term] = operation

            """DATA COLLECTION"""
            
            operation_index += term
        
    def _get_movement_details(self, possible_moves):
        # Determine the number of moves to do, calculated randomly and from 1 to k
        moves = 1 if self.displacements == 1 else random.choice(range(1, min(self.displacements, abs(possible_moves))))
        # Determine the direction of the movement
        term = 1 if possible_moves >= 0 else -1
        
        return moves, term
        
    def _get_operation_details(self, graph, displacements):
        # Select a machine randomly. Note that machine indeces start start at one. 
        machine_index = random.choice(range(1, len(graph)))
        # Get operation displacements for that machine
        machine_displacements = displacements[machine_index]
        # Choose an operation to move randomly
        operation_index = random.choice(range(len(machine_displacements) - 1))
        # Get max number of moves for selected operation
        moves = machine_displacements.get(machine_index)[operation_index]
        
        """DATA COLLECTION"""
        
        return machine_index, operation_index, moves
        
    def _calculate_displacements(graph):
        result = {}
        for machine_id, operations in graph.items():
            possible_displacements = []
            
            for operation_index in range(len(operations)):
                # when it is the first operation
                if operation_index == 0:
                    possible_displacements.append(len(operations) - 1)
                # when it is the last operation
                elif operation_index == len(operations) - 1:
                    possible_displacements.append(-1 * (len(operations) - 1))
                # for all other operatons
                else:
                    move_left = -operation_index
                    move_right = len(operations) - 1 - operation_index
                    possible_displacements.append((move_left, move_right))
                    
            result[machine_id] = possible_displacements
        
    @property
    def k(self):
        return self.__k

    @k.setter
    def k(self, k):
        if k <= 0:
            self.__k = 1
        else:
            self.__k = k
            
    @property
    def displacements(self):
        return self.__displacements

    @displacements.setter
    def displacements(self, displacements):
        if displacements <= 0:
            self.__displacements = 1
        else:
            self.__displacements = displacements
    
    @abstractmethod
    def perturb_solution(self, current_solution):
        pass

In [34]:
class BestMove(KMoves):
    def perturb_solution(self, current_solution):
        original_graph = current_solution.m_graph
        graph = cp.deepcopy(original_graph)
        machine_graph_copy = OrderedDict(graph)
        
        # Calculate the max number of displacements that each operation is capable of having
        # negative numbers mean movements to the left, while positive ones movements to the right
        displacements = self._calculate_displacements(machine_graph_copy)
        # Get the machine index alongside the operation to move and its corresponding move capability
        machine_index, operation_index, moves = self._get_operation_details(machine_graph_copy, displacements)
        # Choose a direction randomly if it is neither the first or the second
        if type(moves) == tuple:
            moves = int(random.choice(moves))
        
        # Find the actual operation index in time for the selected machine
        machine_operations = graph.get(machine_index)
        operation = machine_operations[operation_index]
        operation_index = machine_operations.index(operation)
        
        """DATA COLLECTION"""
        
        moves, term = self._get_movement_details(moves)
        
        """DATA COLLECTION"""
        
        self._do_displacements(graph, moves, term, machine_index, operation_index)
        
        return [graph]

In [39]:
class RandomMove(KMoves):
    """Creates a new neighbor for the JSSP by permuting the current solution once.
    """
    
    def perturb_solution(self, original_graph):
        original_graph = current_solution.m_graph
        graph = cp.deepcopy(original_graph)
        machine_graph_copy = OrderedDict(graph)
        
        # Calculate the max number of displacements that each operation is capable of having
        # negative numbers mean movements to the left, while positive ones movements to the right
        displacements = self._calculate_displacements(machine_graph_copy)
        # Get the machine index alongside the operation to move and its corresponding move capability
        # for each candidate
        candidates = [] # each tupple has: machine_index, operation_index, moves
        for _ in range(k):
            candidates.append(self._get_operation_details(machine_graph_copy, displacements))
        # Choose a direction randomly if it is neither the first or the second for each candidate
        if type(moves) == tuple:
            moves = int(random.choice(moves))
        
        # Find the actual operation index in time for the selected machine
        machine_operations = graph.get(machine_index)
        operation = machine_operations[operation_index]
        operation_index = machine_operations.index(operation)
        
        """DATA COLLECTION"""
        
        moves, term = self._get_movement_details(moves)
        
        """DATA COLLECTION"""
        
        self._do_displacements(graph, moves, term, machine_index, operation_index)
        
        return graph

In [40]:
class FirstMove(KMoves):
    def perturb_solution(self, current_solution):
        pass